In [3]:
import pandas as pd
import time
import re
import random
import os
from tqdm.auto import tqdm
import openai
import backoff
import concurrent.futures
from getpass import getpass

# Configure batch processing
MAX_WORKERS = 10  # Number of parallel requests - adjust based on your API limits
BATCH_SIZE = 50   # Process results in batches for saving
SAMPLE_SIZE = 5000  # Updated to process 5,000 statements

# Configure file paths
INPUT_FILE = "vp_primary_debates_with_hostility.csv"  # Updated input file with existing labels
OUTPUT_FILE = "vp_primary_debates_with_hostility_updated.csv"  # New output file
CHECKPOINT_FILE = "hostility_annotation_progress_updated.csv"  # Updated checkpoint file

# Function with exponential backoff for handling rate limits
@backoff.on_exception(backoff.expo,
                     Exception,  # Catch all exceptions
                     max_tries=3,
                     factor=2)
def classify_hostility(statement, demonstrations, client):
    """Classify hostility with retry logic for rate limits, using demonstrations"""

    # Create a prompt that includes your manual examples as demonstrations
    demo_text = ""
    for demo in demonstrations:
        demo_text += f"Statement: \"{demo['statement']}\"\n"
        demo_text += f"Hostility label: {demo['label']}\n\n"

    prompt = (
        "You are a political analyst. Label the following political debate statement for hostility.\\n\\n"
        "Label definitions:\\n"
        "2 = Very hostile (clearly attacking, highly negative, aggressive)\\n"
        "1 = Somewhat or maybe hostile (mildly negative, critically questioning, passive-aggressive)\\n"
        "0 = Not hostile at all (neutral, informative, collaborative)\\n\\n"
        "Here are some examples of labeled statements:\\n\\n"
        f"{demo_text}"
        f"Statement: \"{statement}\"\\n"
        "Hostility label (just a number):"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Faster than GPT-4
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=10  # We only need a short response
    )

    # Extract the first digit 0, 1, or 2 from the response
    output = response.choices[0].message.content.strip()
    match = re.search(r"\b[012]\b", output)
    if match:
        return int(match.group(0))
    else:
        print(f"Could not parse label from response: {output}")
        return None

def process_statements(statements, indices, demonstrations, client):
    """Process a batch of statements and return results"""
    results = []
    for idx, statement in zip(indices, statements):
        try:
            label = classify_hostility(statement, demonstrations, client)
            results.append((idx, label))
            # Small delay to avoid clustering requests too closely
            time.sleep(0.1)
        except Exception as e:
            print(f"Error on statement {idx}: {str(e)}")
            results.append((idx, None))
    return results

def main():
    # Get API key
    api_key = getpass("Enter your OpenAI API key: ")
    client = openai.OpenAI(api_key=api_key)

    # Load the dataset
    print(f"Loading dataset from {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE)
        print(f"Loaded {len(df)} statements.")
    except FileNotFoundError:
        print(f"Error: {INPUT_FILE} not found.")
        return
    except Exception as e:
        print(f"Error loading CSV: {str(e)}")
        return

    # Extract your manual demonstrations
    print("Extracting manual demonstrations...")
    manual_demos = []
    for idx, row in df[df['My Hostile Rating'].notna()].iterrows():
        manual_demos.append({
            'statement': row['statement'],
            'label': int(row['My Hostile Rating'])
        })

    print(f"Found {len(manual_demos)} manual demonstrations")

    if not manual_demos:
        print("No manual demonstrations found. Please add some manual ratings in the 'My Hostile Rating' column.")
        return

    # Display the demonstrations for verification
    print("\nDemonstrations that will be used:")
    for i, demo in enumerate(manual_demos):
        print(f"{i+1}. Statement: '{demo['statement'][:50]}...' Label: {demo['label']}")

    # Test API connection
    print("\nTesting API connection...")
    try:
        test_result = classify_hostility("This is a test statement.", manual_demos, client)
        print(f"API test successful. Result: {test_result}")
    except Exception as e:
        print(f"API test failed: {str(e)}")
        print("Please check your API key and try again.")
        return

    # Try to load previous progress if available
    try:
        df_annotated = pd.read_csv(CHECKPOINT_FILE)
        print(f"Loaded previous progress with {df_annotated['hostility_label_revised'].notna().sum()} labeled statements")
    except FileNotFoundError:
        print("Starting fresh annotation")
        df_annotated = df.copy()
        if 'hostility_label_revised' not in df_annotated.columns:
            df_annotated['hostility_label_revised'] = None

    # Create a list of indices that still need processing
    # Filter out statements that already have original hostility_label or your manual rating
    unlabeled_mask = (
        (df_annotated['hostility_label'].isna()) &  # No previous algorithm label
        (df_annotated['My Hostile Rating'].isna()) &  # No manual label by you
        (df_annotated['hostility_label_revised'].isna())  # No revised label yet
    )
    remaining_indices = df_annotated[unlabeled_mask].index.tolist()

    # Sample statements if requested
    total_remaining = len(remaining_indices)
    SAMPLE_SIZE = 5000  # Updated to process 5,000 statements

    if total_remaining > SAMPLE_SIZE:
        print(f"Randomly sampling {SAMPLE_SIZE} statements from {total_remaining} completely unlabeled statements")
        selected_indices = random.sample(remaining_indices, SAMPLE_SIZE)
    else:
        print(f"Processing all {total_remaining} remaining unlabeled statements")
        selected_indices = remaining_indices

    # Create batches for processing
    batches = [selected_indices[i:i + BATCH_SIZE] for i in range(0, len(selected_indices), BATCH_SIZE)]
    print(f"Created {len(batches)} batches of up to {BATCH_SIZE} statements each")

    # Process all batches with progress bar
    start_time = time.time()
    processed_count = 0

    with tqdm(total=len(selected_indices), desc="Labeling statements") as pbar:
        for batch_idx, batch_indices in enumerate(batches):
            batch_statements = [df.loc[idx, 'statement'] for idx in batch_indices]

            # Process this batch using parallel workers
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                # Split the batch into smaller chunks for parallel processing
                chunk_size = max(1, len(batch_indices) // MAX_WORKERS)
                chunks_indices = [batch_indices[i:i + chunk_size] for i in range(0, len(batch_indices), chunk_size)]
                chunks_statements = [batch_statements[i:i + chunk_size] for i in range(0, len(batch_statements), chunk_size)]

                # Submit all chunks for parallel processing
                futures = [
                    executor.submit(process_statements, chunk_statements, chunk_indices, manual_demos, client)
                    for chunk_indices, chunk_statements in zip(chunks_indices, chunks_statements)
                ]

                # Collect results as they complete
                batch_results = []
                for future in concurrent.futures.as_completed(futures):
                    batch_results.extend(future.result())

            # Update dataframe with results
            for idx, label in batch_results:
                if label is not None:
                    df_annotated.loc[idx, 'hostility_label_revised'] = label
                    processed_count += 1

            # Save progress after each batch
            df_annotated.to_csv(CHECKPOINT_FILE, index=False)

            # Update progress bar
            pbar.update(len(batch_indices))

            # Calculate and display time statistics
            elapsed_time = time.time() - start_time
            statements_per_second = processed_count / elapsed_time if elapsed_time > 0 else 0
            remaining_batches = len(batches) - (batch_idx + 1)
            estimated_remaining_time = remaining_batches * (elapsed_time / (batch_idx + 1)) if batch_idx > 0 else 0

            print(f"\nBatch {batch_idx+1}/{len(batches)} complete. "
                  f"Speed: {statements_per_second:.2f} statements/sec. "
                  f"Est. remaining time: {estimated_remaining_time/60:.1f} minutes.")

    # Final save
    df_annotated.to_csv(OUTPUT_FILE, index=False)

    # Show summary statistics
    print("\nHostility label distribution for newly labeled statements:")
    hostility_new = df_annotated['hostility_label_revised'].value_counts().sort_index()
    print(hostility_new)

    # Calculate total labeled statements across all methods
    total_labeled = (
        df_annotated['hostility_label'].notna() |
        df_annotated['My Hostile Rating'].notna() |
        df_annotated['hostility_label_revised'].notna()
    ).sum()

    total_statements = len(df_annotated)
    labeled_percent = (total_labeled / total_statements) * 100

    print(f"\nTotal statements in dataset: {total_statements}")
    print(f"Total labeled statements (across all methods): {total_labeled}")
    print(f"Percentage of dataset now labeled: {labeled_percent:.2f}%")

    # Show counts for each labeling method
    print(f"\nStatements labeled by original algorithm: {df_annotated['hostility_label'].notna().sum()}")
    print(f"Statements manually labeled by you: {df_annotated['My Hostile Rating'].notna().sum()}")
    print(f"Statements newly labeled in this run: {df_annotated['hostility_label_revised'].notna().sum()}")

    # Display confusion matrix
    if total_compared > 0:
        print("\nConfusion matrix (original vs revised):")
        confusion = pd.crosstab(
            df_annotated['hostility_label'],
            df_annotated['hostility_label_revised'],
            rownames=['Original'],
            colnames=['Revised'],
            margins=True
        )
        print(confusion)

    labeled_percent = df_annotated['hostility_label_revised'].notna().mean() * 100
    print(f"\nPercentage labeled with revised approach: {labeled_percent:.2f}%")

    total_time = time.time() - start_time
    print(f"\nTotal processing time: {total_time/60:.2f} minutes")
    print(f"Average speed: {processed_count/total_time:.2f} statements per second")
    print(f"Final output saved to: {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

Enter your OpenAI API key: ··········
Loading dataset from vp_primary_debates_with_hostility.csv...
Loaded 45009 statements.
Extracting manual demonstrations...
Found 10 manual demonstrations

Demonstrations that will be used:
1. Statement: 'You've already made two comments, John. It's my tu...' Label: 1
2. Statement: 'Oh, come on, Bernie....' Label: 1
3. Statement: 'Did -- Joe, wait a minute. I'll answer your questi...' Label: 1
4. Statement: 'Well, Wolf, I've just been personally attacked aga...' Label: 1
5. Statement: 'I don't know if you can hear. I can hear. But anyw...' Label: 1
6. Statement: '... You lose by [inaudible] points. She will wipe ...' Label: 1
7. Statement: 'Trump Steaks is gone. You have ruined these compan...' Label: 2
8. Statement: 'Lyin' Ted....' Label: 2
9. Statement: 'I will, and the wall just got 10 feet taller, beli...' Label: 2
10. Statement: 'You're losing so badly you — I want to......' Label: 2

Testing API connection...
API test successful. Result: 0
Sta

Labeling statements:   0%|          | 0/5000 [00:00<?, ?it/s]


Batch 1/100 complete. Speed: 10.43 statements/sec. Est. remaining time: 0.0 minutes.

Batch 2/100 complete. Speed: 11.86 statements/sec. Est. remaining time: 6.9 minutes.

Batch 3/100 complete. Speed: 11.24 statements/sec. Est. remaining time: 7.2 minutes.

Batch 4/100 complete. Speed: 11.97 statements/sec. Est. remaining time: 6.7 minutes.

Batch 5/100 complete. Speed: 11.28 statements/sec. Est. remaining time: 7.0 minutes.

Batch 6/100 complete. Speed: 10.47 statements/sec. Est. remaining time: 7.5 minutes.

Batch 7/100 complete. Speed: 10.79 statements/sec. Est. remaining time: 7.2 minutes.

Batch 8/100 complete. Speed: 11.21 statements/sec. Est. remaining time: 6.8 minutes.

Batch 9/100 complete. Speed: 11.22 statements/sec. Est. remaining time: 6.8 minutes.

Batch 10/100 complete. Speed: 11.61 statements/sec. Est. remaining time: 6.5 minutes.

Batch 11/100 complete. Speed: 10.53 statements/sec. Est. remaining time: 7.0 minutes.

Batch 12/100 complete. Speed: 10.87 statements/sec.

INFO:backoff:Backing off classify_hostility(...) for 1.2s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 0.9s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 0.7s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit re


Batch 30/100 complete. Speed: 12.18 statements/sec. Est. remaining time: 4.8 minutes.


...

INFO:backoff:Backing off classify_hostility(...) for 1.6s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on tokens per min (TPM): Limit 200000, Used 199702, Requested 365. Please try again in 20ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 0.9s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on tokens per min (TPM): Limit 200000, Used 199666, Requested 372. Please try again in 11ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 1.1s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate li


Batch 97/100 complete. Speed: 8.57 statements/sec. Est. remaining time: 0.3 minutes.


INFO:backoff:Backing off classify_hostility(...) for 1.1s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on requests per min (RPM): Limit 500, Used 500, Requested 1. Please try again in 120ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 1.4s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-zq7Modopi4RhdMcJDgnrgDbv on tokens per min (TPM): Limit 200000, Used 199771, Requested 365. Please try again in 40ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})
INFO:backoff:Backing off classify_hostility(...) for 1.0s (openai.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit


Batch 98/100 complete. Speed: 8.50 statements/sec. Est. remaining time: 0.2 minutes.

Batch 99/100 complete. Speed: 8.52 statements/sec. Est. remaining time: 0.1 minutes.

Batch 100/100 complete. Speed: 8.49 statements/sec. Est. remaining time: 0.0 minutes.

Hostility label distribution for newly labeled statements:
hostility_label_revised
0    3897
1    1036
2      60
Name: count, dtype: int64

Total statements in dataset: 45009
Total labeled statements (across all methods): 9005
Percentage of dataset now labeled: 20.01%

Statements labeled by original algorithm: 4002
Statements manually labeled by you: 10
Statements newly labeled in this run: 4993


NameError: name 'total_compared' is not defined

In [2]:
pip install backoff